# Dataset build

## Segmentation

In [1]:
!python3 -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone https://github.com/facebookresearch/detectron2.git
dist = distutils.core.run_setup("./detectron2/setup.py")
!python3 -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# python -m pip install git+https://github.com/facebookresearch/detectron2.git

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import matplotlib.pyplot as plt

# import some common libraries
import pandas as pd
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import pycocotools.mask as mask_util


def binary_mask_to_polygons(mask):
    contours, hierarchy = cv2.findContours(mask.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for contour in contours:
        if len(contour) >= 3:  # valid polygon
            polygon = contour.flatten().tolist()
            reshaped_polygon = [[polygon[i], polygon[i+1]] for i in range(0, len(polygon), 2)]
            polygons.append(reshaped_polygon)
    return polygons

def save_json(file_name, outputs, output_folder):
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.tolist()
    pred_boxes = instances.pred_boxes.tensor.tolist()
    pred_scores = instances.scores.tolist()
    pred_masks = instances.pred_masks.numpy()
    pred_polygons = [binary_mask_to_polygons(mask) for mask in pred_masks]

    detection_results = []
    for cls, box, score, polygons in zip(pred_classes, pred_boxes, pred_scores, pred_polygons):
        instance_result = {
            "class": cls,
            "box": box,
            "score": score,
            "mask": [{"polygon": polygon} for polygon in polygons]
        }
        detection_results.append(instance_result)

    result_dict = {
        "file_name": file_name,
        "instances": detection_results
    }

    json_output_file_path = os.path.join(output_folder, f"{file_name.split('.')[0]}.json")
    
    os.makedirs(os.path.dirname(json_output_file_path), exist_ok=True)

    with open(json_output_file_path, 'w') as json_file:
        json.dump(result_dict, json_file, indent=4)


def save_test_image(image, file_name, output_folder):
    output_image_path = os.path.join(output_folder, f"test_{file_name}")
    cv2.imwrite(output_image_path, image)

def visualize_single_image(image_path, cfg, predictor, output_folder):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out_img = out.get_image()[:, :, ::-1]  # Convert RGB to BGR for OpenCV

    plt.imshow(out_img)
    plt.show()
    
    # Save visualized image
    file_name = os.path.basename(image_path)
    save_test_image(out_img, file_name, output_folder)
    
    # Save JSON
    save_json(f"test_{file_name}", outputs, output_folder)

def process_folder(input_folder, output_folder, cfg, predictor):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            im = cv2.imread(file_path)  # Read image in BGR format
            if im is None:
                continue
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            outputs = predictor(im_rgb)  # Get predictions from the model

            v = Visualizer(im_rgb, metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.5)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            out_img = out.get_image()[:, :, ::-1]  # Convert RGB to BGR for OpenCV

            test_name = f"test_{file_name}"
            
            # Save IMAGE
            save_test_image(out_img, file_name, output_folder)

            # Save JSON
            save_json(test_name, outputs, output_folder)


fatal: destination path 'detectron2' already exists and is not an empty directory.
matplotlib data path: /Users/holmes/anaconda3/envs/test/lib/python3.11/site-packages/matplotlib/mpl-data
CONFIGDIR=/Users/holmes/.matplotlib
interactive is False
platform is darwin
CACHEDIR=/Users/holmes/.matplotlib
Using fontManager instance from /Users/holmes/.matplotlib/fontlist-v330.json
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [3]:
cfg = get_cfg()
cfg.merge_from_file("/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/config.yaml")
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/model_final.pth"


predictor = DefaultPredictor(cfg)

# # ! Single test
# image_path = r"/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/Model_18_0025.jpeg"
# output_folder = f"/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/Output"
# visualize_single_image(image_path, cfg, predictor, output_folder)

! Folder test
input_folder = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/1-input_reg'
output_folder = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/2-output_seg'
output_json_folder = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/3-output_json'
process_folder(input_folder, output_folder, cfg, predictor)

Loading config /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[07/25 21:47:05 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/model_final.pth ...
[Checkpointer] Loading from /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/model_final.pth ...
zsh:1: command not found: Folder


## Selection the max area

In [ ]:
#FUNCTION: Select the mask with the largest area using Shoelace formula
def polygon_area(polygon):
    polygon = np.array(polygon)
    x = polygon[:, 0]
    y = polygon[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def draw_polygon_on_image(image, polygon):
    pts = np.array(polygon, np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=3)
    return image

max_area = 0
max_area_index = -1

input_folder = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/1-input_reg'
output_folder = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/2-output_seg'
output_json = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/3-output_json'
output_max = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Transformation/4-output_max'


#FUNCTION：load JSON
#FIXME: check if the file is a json file
for each file endswith('.json') in output_folder:
    seg_output = json.load(f)

    #FUNCTION： save the name of original json
    json_name = os.path.basename(file)

    max_area = 0
    max_area_index = -1

    #FUNCTION: loop the masks and select the max area
    for i, instance in enumerate(seg_output['instances']):
        polygons = instance['mask']
        for j, polygon_data in enumerate(polygons):
            polygon = polygon_data['polygon']
            area = polygon_area(polygon)
            print(f"Mask_{i}_{j} area: {area}")
            if area > max_area:
                max_area = area
                max_area_index = f"Mask_{i}_{j}"
                max_polygon = polygon

        #FUNCTION: save the max mask as json
        save_json(json_name, max_polygon, output_json)
        
        #FUNCTION: save the image with max mask
        image_with_polygon = draw_polygon_on_image(image_org.copy(), max_polygon)
        save_test_image(image_with_polygon, json_name, output_max)